<a href="https://colab.research.google.com/github/gabPetti/CVision/blob/main/CVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CVision

## 🚀 Visão Geral

O CVision é uma ferramenta de PLN que atua como um consultor de RH digital. Em vez de uma análise de currículo genérica, este projeto foca em resolver um problema do mundo real que toda pessoa que já entrou no mercado de trabalho já se perguntou: "Meu currículo está bom para esta vaga?"

O usuário fornece dois inputs de texto:
- O texto completo do seu Currículo (CV).
- O texto completo da Descrição da Vaga (ex: copiado do LinkedIn, Vagas.com).

A ferramenta, então, gera uma análise de aderência (Gap Analysis) tática, informando os pontos fortes, as lacunas (o que falta) e sugestões estratégicas para o candidato se destacar.

## ✨ Features
- **Gap Analysis**: Compara o CV do usuário com os requisitos da vaga.
- **Extração de Requisitos**: Identifica e extrai automaticamente as hard skills e soft skills mais importantes da descrição da vaga.
- **Relatório Tático**: Gera um relatório simples em markdown com:
  - Pontos Fortes: Onde o seu CV brilha para esta vaga.
  - Pontos de Melhoria: Quais requisitos da vaga não estão claros no seu CV.
  - Sugestão Estratégica: Uma dica de ouro para destacar na sua carta de apresentação ou entrevista.

## 🛠️ Arquitetura e Uso Criativo do LangChain

Aqui está o núcleo do projeto, utilizando o LangChain e múltiplas técnicas de PLN para alcançar as features descritas acima.

Em vez de um prompt único, o projeto usa a LCEL (LangChain Expression Language) para orquestrar um pipeline de duas etapas que simula o raciocínio de um recrutador. A pipeline consiste na utilização das tecnicas de Extração de Entidades encadeado de Sumarização Comparativa.

Cadeia 1: Extração de Requisitos
- Input: O `texto_vaga`.
- Processo: O texto passa por um `PromptTemplate` que instrui o LLM a atuar como um "Tech Recruiter".
- Técnica: O LLM (Gemini) extrai as skills essenciais.
- Output: Um `JsonOutputParser` força o LLM a retornar um JSON estruturado com as `hard_skills` e `soft_skills`.

Cadeia 2: Análise de Aderência
- Input: O texto_cv do usuário + o JSON gerado pela Cadeia 1.
- Processo: Um PromptTemplate instrui o LLM a atuar como "Consultor de Carreira".
- Técnica: O LLM (Gemini) recebe ambos os inputs e deve comparar os dois, gerando a análise de aderência (gap analysis).
- Output: Um `StrOutputParser` retorna o relatório final em texto (Markdown).

O RunnablePassthrough do LangChain é usado para gerenciar e rotear esses múltiplos inputs (CV, Vaga) através do pipeline de forma eficiente.

## ✅ Atendimento aos Critérios de Avaliação
- Uso do LangChain: Uso da LCEL para orquestrar um pipeline sequencial, gerenciar múltiplos inputs e usar parsers.
- Uso de um LLM: Uso do Gemini 2.5 Flash para ambas as etapas de PLN.
- Uso de Página Web (Corpus): O corpus é o texto de uma vaga de emprego real (ex: LinkedIn, Vagas.com), um dado não-estruturado do "mundo real" que o usuário fornece.
- Github: O projeto está disponível [link-para-seu-github-aqui].
- Criatividade: A criatividade reside na arquitetura de duas etapas, que simula o raciocínio de um recrutador (primeiro entende a vaga, depois analisa o CV), ao invés de usar um prompt único e simplista. Este projeto exala criatividade e inovação


## 🔧 Tecnologias Utilizadas
- Linguagem: Python
- Plataforma: Google Colab
- Framework: LangChain (LCEL, langchain-google-genai, langchain-core)
- LLM: Google Gemini 2.5 Flash

## Codigo

### 1. Instação de dependencias

In [ ]:
!pip install langchain-google-genai langchain-core

### 2. Configuração da API Key

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

### 3. Instanciação

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

llm = ChatGoogleGenerativeAI(
    temperature = 0,
    model = "gemini-2.5-flash",
    api_key = GOOGLE_API_KEY
)
parser_json = JsonOutputParser()
parser_str = StrOutputParser()

### 4. Dados de Exemplo

In [ ]:
texto_vaga_exemplo = """
## Vaga de Desenvolvedor(a) Python Sênior

Estamos procurando por um(a) Desenvolvedor(a) Python Sênior experiente para se juntar ao nosso time dinâmico. Você será responsável por desenvolver e manter aplicações web escaláveis, trabalhar com bancos de dados relacionais (PostgreSQL) e NoSQL (MongoDB), e implementar soluções de CI/CD (Jenkins, Docker).

**Requisitos:**

*   Experiência comprovada com Python e frameworks web como Django ou Flask.
*   Sólidos conhecimentos em SQL e bancos de dados PostgreSQL.
*   Experiência com MongoDB.
*   Familiaridade com metodologias Ágeis (Scrum).
*   Experiência com ferramentas de CI/CD (Jenkins, Docker).
*   Habilidade de trabalhar em equipe.
*   Excelente comunicação verbal e escrita.
*   Capacidade de resolver problemas complexos.
"""

texto_cv_exemplo = """
## João Silva

**Resumo:** Desenvolvedor Fullstack com 5 anos de experiência em desenvolvimento web.

**Experiência:**

*   **Empresa X** - Desenvolvedor Python Pleno (2 anos): Desenvolvimento de APIs RESTful com Flask, integração com PostgreSQL. Participação em rituais Scrum.
*   **Empresa Y** - Desenvolvedor Web (3 anos): Desenvolvimento de front-end com React e back-end com Node.js.

**Habilidades:**

*   **Hard Skills:** Python, Flask, PostgreSQL, JavaScript, React, Node.js, Git.
*   **Soft Skills:** Trabalho em equipe, Proatividade.

**Educação:** Bacharelado em Ciência da Computação.
"""

### 5. Extração de Requisitos

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_extracao = ChatPromptTemplate.from_template("""
Você é um 'Tech Recruiter' sênior. Analise a descrição da vaga abaixo.
Extraia as 5 'hard_skills' (ferramentas, linguagens) e as 3 'soft_skills' (comportamentos) mais essenciais.
Retorne APENAS um objeto JSON com as chaves "hard_skills" e "soft_skills".

Vaga:
{texto_vaga}
""")

chain_extracao = prompt_extracao | llm | parser_json

input_variables=['texto_vaga'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['texto_vaga'], input_types={}, partial_variables={}, template='\nVocê é um \'Tech Recruiter\' sênior. Analise a descrição da vaga abaixo.\nExtraia as 5 \'hard_skills\' (ferramentas, linguagens) e as 3 \'soft_skills\' (comportamentos) mais essenciais.\nRetorne APENAS um objeto JSON com as chaves "hard_skills" e "soft_skills".\n\nVaga:\n{texto_vaga}\n'), additional_kwargs={})]


### 6. Análise de Aderência

In [ ]:
prompt_analise = ChatPromptTemplate.from_template("""
Você é um 'Consultor de Carreira'.
Faça uma análise de aderência (Gap Analysis) em Markdown comparando o CV do candidato com os requisitos da vaga.

Seja tático e gere um relatório com:
- **Pontos Fortes**: Onde o CV atende aos requisitos.
- **Pontos de Melhoria**: Quais requisitos da vaga faltam no CV.
- **Sugestão Estratégica**: Uma dica de ouro para a carta de apresentação.

**Requisitos da Vaga (JSON):**
{requisitos}

**Currículo (CV) do Candidato:**
{cv}
""")

chain_analise = prompt_analise | llm | parser_str

### 7. Orquestração do LangChain

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# O input da cadeia principal será um dict: {"vaga": ..., "cv": ...}
# Usamos `RunnablePassthrough.assign` ou lambdas para rotear os inputs

main_chain = {
    "requisitos": (lambda x: x["vaga"]) | chain_extracao,
    "cv": (lambda x: x["cv"])
} | prompt_analise | llm | parser_str

### 8. Execução do Projeto

In [ ]:
from IPython.display import display, Markdown

input_data = {"vaga": texto_vaga_exemplo, "cv": texto_cv_exemplo}

print("Executando CVision... Gerando análise de aderência...")
resultado = main_chain.invoke(input_data)

print("\n--- RELATÓRIO DE ANÁLISE TÁTICA ---")
display(Markdown(resultado))

Executando CVision... Gerando análise de aderência...

--- RELATÓRIO DE ANÁLISE TÁTICA ---


Olá, João! Como seu Consultor de Carreira, realizei uma análise detalhada do seu currículo em comparação com os requisitos da vaga. O objetivo é identificar onde você brilha e onde podemos fortalecer sua candidatura.

---

# Análise de Aderência (Gap Analysis) - Currículo vs. Requisitos da Vaga

## Perfil do Candidato: João Silva

*   **Resumo:** Desenvolvedor Fullstack com 5 anos de experiência.
*   **Experiência:**
    *   Empresa X (2 anos): Desenvolvedor Python Pleno (Flask, PostgreSQL, Scrum).
    *   Empresa Y (3 anos): Desenvolvedor Web (React, Node.js).
*   **Habilidades:** Python, Flask, PostgreSQL, JavaScript, React, Node.js, Git, Trabalho em equipe, Proatividade.
*   **Educação:** Bacharelado em Ciência da Computação.

## Requisitos da Vaga:

*   **Hard Skills:** Python, Django/Flask, PostgreSQL, MongoDB, Docker
*   **Soft Skills:** Trabalho em Equipe, Comunicação, Resolução de Problemas

---

## Análise Detalhada:

### Pontos Fortes (Onde o CV atende aos requisitos)

1.  **Python:** Sua experiência como "Desenvolvedor Python Pleno" e a listagem explícita da habilidade demonstram um forte alinhamento.
2.  **Flask:** A menção de "Desenvolvimento de APIs RESTful com Flask" na Empresa X e a listagem em suas habilidades confirmam experiência direta com um dos frameworks solicitados (Flask é uma alternativa a Django).
3.  **PostgreSQL:** Sua experiência de "integração com PostgreSQL" na Empresa X e a listagem em habilidades mostram domínio desta base de dados relacional.
4.  **Trabalho em Equipe:** A "Participação em rituais Scrum" na Empresa X e a listagem explícita em suas soft skills são evidências claras de sua capacidade de trabalhar em equipe.

### Pontos de Melhoria (Quais requisitos da vaga faltam no CV)

1.  **MongoDB:** Não há menção de experiência ou familiaridade com MongoDB no seu currículo.
2.  **Docker:** Não há menção de experiência ou familiaridade com Docker no seu currículo.
3.  **Comunicação:** Embora o trabalho em equipe e Scrum impliquem comunicação, a soft skill "Comunicação" não é explicitamente listada ou detalhada em seu CV.
4.  **Resolução de Problemas:** Esta soft skill crucial não é explicitamente mencionada ou ilustrada com exemplos em seu currículo.

---

## Sugestão Estratégica para a Carta de Apresentação:

João, sua carta de apresentação é a sua chance de não apenas reforçar seus pontos fortes, mas também de abordar proativamente os pontos de melhoria, transformando-os em oportunidades.

**Dica de Ouro:**

1.  **Reafirme seus Pontos Fortes com Entusiasmo:** Comece conectando diretamente sua experiência com Python, Flask e PostgreSQL aos requisitos da vaga. Mencione como sua experiência em construir APIs e integrar bancos de dados se alinha perfeitamente com o que eles procuram. Expresse seu entusiasmo pela oportunidade e como sua experiência em trabalho em equipe (Scrum) o torna um candidato colaborativo.

2.  **Aborde os Gaps de Hard Skills com Proatividade e Aprendizado:**
    *   **MongoDB/Docker:** Não minta, mas mostre proatividade. Você pode dizer algo como: "Embora minha experiência direta tenha sido primariamente com PostgreSQL, minha capacidade de adaptação e aprendizado rápido, demonstrada na transição entre tecnologias (ex: React/Node.js na Empresa Y), me deixa confiante para rapidamente adquirir proficiência em MongoDB e Docker, tecnologias que vejo como cruciais no desenvolvimento moderno e que tenho grande interesse em dominar." Ou, se tiver estudado por conta própria, "Tenho explorado ativamente [MongoDB/Docker] em projetos pessoais e estou animado para aplicar e aprofundar esse conhecimento em um ambiente profissional."

3.  **Ilustre as Soft Skills Faltantes com Exemplos Concretos:**
    *   **Comunicação:** Pense em situações onde sua comunicação foi fundamental. "Minha participação ativa em rituais Scrum na Empresa X me permitiu aprimorar minhas habilidades de comunicação, garantindo alinhamento claro de expectativas e progresso com a equipe e stakeholders."
    *   **Resolução de Problemas:** Esta é uma habilidade que todo desenvolvedor usa. Pense em um desafio técnico que você superou. "Ao longo dos meus 5 anos de experiência, enfrentei e resolvi diversos desafios técnicos complexos, como [mencione brevemente um exemplo], o que aprimorou significativamente minha capacidade de análise crítica e resolução de problemas de forma eficaz."

**Estrutura da Mensagem:**

*   **Parágrafo 1:** Expressar entusiasmo pela vaga, destacar os 5 anos de experiência e os pontos fortes (Python, Flask, PostgreSQL) que se alinham diretamente.
*   **Parágrafo 2:** Focar nos resultados. Como sua experiência em Flask/PostgreSQL gerou valor? (Ex: "Desenvolvi APIs robustas que melhoraram X ou Y").
*   **Parágrafo 3:** Abordar os pontos de melhoria (MongoDB/Docker) com uma postura de aprendizado e proatividade.
*   **Parágrafo 4:** Destacar as soft skills (Comunicação, Resolução de Problemas) com exemplos práticos e reforçar o "Trabalho em Equipe".
*   **Conclusão:** Reafirmar o interesse, a adequação ao perfil e a vontade de contribuir com a equipe.

Ao seguir esta abordagem, você não só apresentará um perfil forte e alinhado, mas também demonstrará autoconsciência, proatividade e um desejo genuíno de crescer e aprender, qualidades altamente valorizadas por recrutadores.

Boa sorte, João!